In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()


In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, col, round, avg
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), header=True)
df.show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [6]:
# turn price into float
df = df.withColumn('price',col('price').cast('float'))

In [7]:
# turn sqft_living into float
df = df.withColumn('sqft_living',col('sqft_living').cast('float'))

In [8]:
# turn view into float
df = df.withColumn('view',col('view').cast('float'))

In [9]:
df_with_year = df.withColumn('year', year(df['date']))
df_with_year.show(10)

+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923.0|       4|        3|     3167.0|   11733|     2|         1|76.0|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628.0|       2|        2|     2235.0|   14384|     1|         0|23.0|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866.0|       2|        2|     2127.0|   10575|     2|         0| 0.0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895.0|       2|        2|     1631.0|   11149|     2|         0| 0.0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418.0|       3|        2|     2249.0|   13878|     2|         0| 4.0|2022|
|5aa00529-0533-46b...|20

In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# create a column 'year' pulled from the 'date' column
df_with_year = df.withColumn('year', year(df['date']))

df_with_year.filter(df_with_year['bedrooms']==4).groupBy('year').agg(round(avg('price'),2).alias('avg_price')).show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
df_with_year.filter((df_with_year['bedrooms']==3)&(df_with_year['bathrooms']==3)).groupby('year').agg(round(avg('price'),2).alias('avg_price')).show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|292725.69|
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
+----+---------+



In [16]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
df_with_year.filter((df_with_year['bedrooms']==3)&(df_with_year['bathrooms']==3)&(df_with_year['floors']==2)
                    &(df_with_year['sqft_living'] >=2000)).groupby('year').agg(round(avg('price'),2).alias('avg_price')).show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|290242.99|
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
+----+---------+



In [31]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

filtered_df = df[df['price'] >= 350000]
average_view = round(filtered_df['view'].mean(), 2)
average_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

TypeError: 'Column' object is not callable

In [32]:
# 7. Cache the the temporary table home_sales.
df.cache()

DataFrame[id: string, date: string, date_built: string, price: float, bedrooms: string, bathrooms: string, sqft_living: float, sqft_lot: string, floors: string, waterfront: string, view: float]

In [33]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

False

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()




print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 


In [ ]:
# 11. Read the formatted parquet data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

